In [140]:
import pandas as pd
import numpy as np
import statsmodels.stats as sm
import scipy
from scipy.optimize import linprog

In [141]:
stargas_entire = pd.ExcelFile("StarGas - All Traffic Sources.xlsx")

In [142]:
stargas = stargas_entire.parse('abc')

In [143]:
print(stargas.head())

  Traffic_Source MediaType       Date                             Device  \
0        Adwords       SEM 2016-01-01                          Computers   
1        Adwords       SEM 2016-01-01  Mobile devices with full browsers   
2        Adwords       SEM 2016-01-01         Tablets with full browsers   
3        Adwords       SEM 2016-01-02                          Computers   
4        Adwords       SEM 2016-01-02  Mobile devices with full browsers   

   Clicks  Impressions  Actual_Spend  Conversions Traffic_Source_Final  
0     124        15696        389.22          7.0              Adwords  
1     195        10540        643.87         11.0              Adwords  
2      67         4468        220.83          4.0              Adwords  
3     181        19544        537.48         11.0              Adwords  
4     306        14584       1062.53          9.0              Adwords  


In [144]:
aggregations = {'Clicks': 'sum',
               'Impressions': 'sum',
               'Actual_Spend': 'sum',
               'Conversions': 'sum'}

In [145]:
stargas_agg = stargas[(stargas['Date'] >= '2017-01-01')].groupby('Traffic_Source_Final').agg(aggregations)

In [146]:
stargas_agg['ClicksperSpend'] = stargas_agg['Clicks']/stargas_agg['Actual_Spend']
stargas_agg['ImpressionsperSpend'] = stargas_agg['Impressions']/stargas_agg['Actual_Spend']
stargas_agg['ConversionsperSpend'] = stargas_agg['Conversions']/stargas_agg['Actual_Spend']

In [147]:
###Reading in the constraints file
constraints = pd.read_csv('Constraints.txt',sep='|')
print(constraints.head())

    Description  Adwords  Bing  DCM  Facebook Sign  Quantity
0  Total_Budget        1     1    1         1   <=   1000000
1         Spend        1     0    0         0   <=     10000
2         Spend        0     1    0         0   <=     10000
3         Spend        0     0    1         0   <=     10000
4         Spend        0     0    0         1   <=     10000


In [148]:
c = -1*np.array(stargas_agg['ConversionsperSpend'])
A = [[1.0,1.0,1.0,1.0],[-10.6287265,0.0,0.0,0.0],[0.0,-14.7469492,0.0,0.0],[0.0,0.0,-87.0776054,0.0],[0.0,0.0,0.0,-43.6720415],
    [-0.20812002,0.0,0.0,0.0],[0.0,-0.27602498,0.0,0.0],[0.0,0.0,-0.04120044,0.0],[0.0,0.0,0.0,-0.49814672],[-0.01649546,0.0,0.0,0.0],[0.0,-0.00673206,0.0,0.0],[0.0,0.0,-0.00210477,0.0],[0.0,0.0,0.0,-0.00554307]]
b= [1000000.0,-1000000.0,-1000000.0,-1000000.0,-100000.0,-500.0,-500.0,-500.0,-500.0,-100.0,-100.0,-100.0,-100.0]

In [149]:
x1_bounds=[constraints['Quantity'][1],None]
x2_bounds=[constraints['Quantity'][2],None]
x3_bounds=[constraints['Quantity'][3],None]
x4_bounds=[constraints['Quantity'][4],None]

In [150]:
###Inequality Constraints
global A_ub, b_ub
A_ub = np.ones((1,4))
def inequality_matrix_construction(no_of_channels, stargas_agg, parameter):
    global A_ub
    type_spend = parameter
    arr = np.zeros((4,4))
    for i in range(no_of_channels):
        arr[i][i] = -1*stargas_agg[type_spend][i]
    print(arr)
    A_ub = np.concatenate((A_ub,arr),axis = 0)


inequality_matrix_construction(4, stargas_agg, 'ImpressionsperSpend')
inequality_matrix_construction(4, stargas_agg, 'ClicksperSpend')
inequality_matrix_construction(4, stargas_agg, 'ConversionsperSpend')


all_constraints = list(constraints['Quantity'])
b_ub = np.array(all_constraints[:1] + all_constraints[5:])
b_ub[1:] = -1* b_ub[1:]

[[-10.08597697   0.           0.           0.        ]
 [  0.         -14.33611238   0.           0.        ]
 [  0.           0.         -84.44362265   0.        ]
 [  0.           0.           0.         -42.84069111]]
[[-0.22726002  0.          0.          0.        ]
 [ 0.         -0.29008897  0.          0.        ]
 [ 0.          0.         -0.03659837  0.        ]
 [ 0.          0.          0.         -0.4815473 ]]
[[-0.01905944  0.          0.          0.        ]
 [ 0.         -0.00671416  0.          0.        ]
 [ 0.          0.         -0.00191835  0.        ]
 [ 0.          0.          0.         -0.00737767]]


In [151]:
res = linprog(c,A_ub=A_ub,b_ub=b_ub,bounds=(x1_bounds,x2_bounds,x3_bounds,x4_bounds),options={"disp": True},method='interior-point')

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 0.9649303694421     -                1.0                 -490.6295194145     
5.116575112794e-05  5.116576390531e-05  3.534656615456e-05  0.9999488342489  5.116574897709e-05  -1109.506602649     
9.658813360566e-07  9.658815772614e-07  6.672546847275e-07  0.9811312391947  9.65881295454e-07   -5952.981539254     
4.813103875546e-07  4.8131050775e-07    3.32501104372e-07   0.5382575877893  4.813103673219e-07  -15631.74874062     
1.385676867244e-09  1.385677213283e-09  9.572598408392e-10  0.9981599388312  1.385676808995e-09  -17708.2840551      
6.974271318535e-14  6.974323151928e-14  4.818367926873e-14  0.9999500910239  6.974322070846e-14  -17823.69783617     
3.525685613193e-18  3.487880667216e-18  0.0                 0.999949989848   3.487161176713e-18  -17823.7036278      
Optimization terminated successfully.


In [152]:
scipy.optimize.OptimizeResult

scipy.optimize.optimize.OptimizeResult

In [153]:
print(res.x)

[919423.60586158  14893.88729571  52128.09249189  13554.4143441 ]


In [156]:
total = np.sum(-1*(res.x)*np.transpose(c))

In [154]:
print(res)

     con: array([], dtype=float64)
     fun: -17823.703627800158
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([6.72170427e-06, 9.17328531e+06, 1.13520442e+05, 4.30188497e+06,
       4.80680478e+05, 2.08448229e+05, 3.82055249e+03, 1.40780347e+03,
       6.02709163e+03, 1.74237036e+04, 1.94515195e-08, 1.21552119e-08,
       1.08681206e-08])
  status: 0
 success: True
       x: array([919423.60586158,  14893.88729571,  52128.09249189,  13554.4143441 ])


In [157]:
total

17823.703627800158